In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import sys
sys.path.append('/home/azureuser/cloudfiles/code/Users/leo900527')

from visualize import *

In [3]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

data_asset = ml_client.data.get("failure_sessions_data", version="1.0")
failure_sessions = pd.read_csv(data_asset.path, parse_dates=["start_time", "end_time"])

data_asset = ml_client.data.get("inverter_data", version="1")
inverter_data = pd.read_parquet(data_asset.path)

failure_sessions['duration'] = pd.to_timedelta(failure_sessions['duration'])
inverter_data['event_local_time'] = pd.to_datetime(inverter_data['event_local_time'])

failure_duration_threshold = pd.Timedelta('3 days')
failure_sessions = failure_sessions[failure_sessions['duration'] > failure_duration_threshold]
print(f"Filtering failure sessions longer than {failure_duration_threshold}...")

Found the config file in: /config.json


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration
Filtering failure sessions longer than 3 days 00:00:00...


In [4]:
metric_type = ["voltage", "current", "power", "energy", 
               'electrical', 'status_metrics',
               'insulation','misc']

metric_columns = [voltage_metrics, current_metrics, power_metrics
                  , energy_metrics, electrical_params, status_metrics,
                  insulation_metrics, misc_metrics ]

device_names = inverter_data['device_name'].unique().tolist()
device_names.sort()

In [9]:
import os

for device_name in device_names:
    n_sessions = failure_sessions[failure_sessions['device_name'] == device_name].shape[0]
    if n_sessions == 0:
        print(f"No failure sessions found for device {device_name}. Skipping...")
        continue
    
    for session in range(n_sessions):
        
        print(f"Visualizing session {session} for device {device_name}...")
        # Create directory for the device if it doesn't exist
        folder_path = f"outputs/metrics_dashboard/{device_name.replace(' ', '_')}/session_{session}"
        os.makedirs(folder_path, exist_ok=True)

        # Visualize each metric type
        for m_type, m_c in zip(metric_type, metric_columns):
            file_path = os.path.join(folder_path, f"{m_type}.html")

            fig = visualize_failure_sessions(inverter_data, failure_sessions, metric_columns=m_c, device_name=device_name, session_to_visualize=session)
            fig.write_html(file_path, full_html=True, include_plotlyjs='cdn')

Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 0 for device INV 51...
Visualizing session 1 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 1 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 1 for device INV 51...
Device INV 51 has 3 failure sessions.
Visualizing session 1 for device INV 51...
Device INV 51 has 3 fai

In [10]:
import shutil

# 將整個 outputs 資料夾壓縮成 zip 檔案
shutil.make_archive('metrics_dashboard', 'zip', 'outputs/metrics_dashboard')

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/leo9005271/code/Users/leo900527/metrics_dashboard.zip'